In [1]:
import numpy as np
from mpg.games import mpg
import networkx as nx

In [24]:
G=mpg.mpg_from_file("data/test01.in")
n=len(G)
A=nx.adjacency_matrix(G,weight=None,nodelist=range(n)).todense()
W=nx.adjacency_matrix(G,nodelist=range(n)).todense()
P=A/A.sum(axis=-1,keepdims=True)

In [25]:
S0,S1=mpg.optimal_strategy_pair(G)
Z0=np.zeros([n,n])
Z1=np.zeros([n,n])
for S,Z in zip([S0,S1],[Z0,Z1]):
    for u in S:
        Z[u,S[u]]=1
Z0,Z1

(array([[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.]]))

In [26]:
W

array([[ 0,  5,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  0,  4,  0,  0,  0],
       [ 0,  0,  0, -7,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  5,  0],
       [ 0,  0,  0,  0,  0, -3,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  3,  0],
       [ 0, -3,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [19]:
G=mpg.MeanPayoffGraph()
G.add_nodes_from(np.arange(3))
G.add_weighted_edges_from([(0,1,100),(0,2,-100),(1,1,2),(2,2,-3),(0,0,5000)])
n=len(G)
A=nx.adjacency_matrix(G,weight=None,nodelist=range(n)).todense()
W=nx.adjacency_matrix(G,nodelist=range(n)).todense()
P=A/A.sum(axis=-1,keepdims=True)

In [6]:
def expected_mean_payoffs(A,W,iters=500):
    E=np.zeros(A.shape[0])
    Z=(A*W).sum(axis=0)
    for k in range(iters):
        E=A.T*E+A.T        
    return E@Z/iters

In [7]:
def expected_mean_payoffs_2(A,W):
    return np.linalg.pinv(np.eye(A.shape[0])-np.linalg.pinv(A))@np.linalg.pinv(A)@(-A*W).sum(axis=0)

In [8]:
from numba import jit

def expected_mean_payoffs_3(A,W,iters=1000):
    n=A.shape[0]
    E=np.zeros(n)
    V=np.arange(n)
    for i in range(iters):
        for k in range(n):
            s=np.random.choice(n,p=A[V[k],:])
            E[k]+=W[V[k],s]
            V[k]=s
    return E/iters

In [308]:
def expected_mean_payoffs_4(W,S0,S1,iters=1000,tol=1e-6):
    R=[]
    for d0,d1 in ([S1,S0],[S0,S1]):
        n=len(W)
        d=d0@d1
        a=np.sum(W*d0,axis=-1)
        b=np.sum(W*d1,axis=-1)
        q=d0@b+a
        D=np.eye(n)
        S=np.zeros([n,n])
        if iters is not None:
            for i in range(iters):
                S+=D
                D=D@d
            R.append((S@q)/(2*iters))
        else:
            D,V=np.linalg.eig(d)
            D_mean=np.where(np.abs(D-1)<=tol,1,0)
            Q=V@np.diag(D_mean)@np.linalg.inv(V)
            R.append(Q@q/2)
    return R

In [309]:
expected_mean_payoffs_4(W,Z0,Z1,None)

[array([ 0.25, -0.5 ,  0.25, -0.5 ,  0.25, -0.5 ,  0.25,  0.25]),
 array([-0.5 ,  0.25, -0.5 ,  0.25, -0.5 ,  0.25, -0.5 , -0.5 ])]

In [7]:
import tensorflow as tf
def expected_mean_payoffs_5(W,S0,S1,iters=1000,tol=1e-6):
    R=[]
    for d0,d1 in ([S1,S0],[S0,S1]):
        d=d0@d1
        a=tf.reduce_sum(W*d0,axis=-1,keepdims=True)
        b=tf.reduce_sum(W*d1,axis=-1,keepdims=True)
        q=d0@b+a
        D=tf.eye(W.shape[-1],batch_shape=W.shape[:-2])
        S=tf.zeros(W.shape)
        print(S.shape)
        print(D.shape)
        if iters is not None:
            for i in range(iters):
                S+=D
                D=D@d
            R.append((S@q)/(2*iters))
        else:
            D,V=tf.linalg.eig(d)
            D_mean=tf.where(tf.abs(D-1)<=tol,tf.constant([1.+0.j],dtype=tf.complex64),tf.constant([0.+0.j],dtype=tf.complex64))
            Q=tf.math.real(V@tf.linalg.diag(D_mean)@tf.linalg.inv(V))
            R.append(Q@q/2)
    return tf.concat(R,axis=-1)
def stack_times(S,n):
    return tf.stack([tf.cast(S,tf.float32)]*n)
expected_mean_payoffs_5(stack_times(W,3),stack_times(Z0,3),stack_times(Z1,3),2000)

2023-04-19 05:31:17.191932: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 05:31:20.108512: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


(3, 8, 8)
(3, 8, 8)
(3, 8, 8)
(3, 8, 8)


<tf.Tensor: shape=(3, 8, 2), dtype=float32, numpy=
array([[[ 0.25125, -0.49875],
        [-0.5    ,  0.25   ],
        [ 0.2495 , -0.5    ],
        [-0.5    ,  0.2505 ],
        [ 0.25   , -0.499  ],
        [-0.5    ,  0.25   ],
        [ 0.24925, -0.50075],
        [ 0.25   , -0.5    ]],

       [[ 0.25125, -0.49875],
        [-0.5    ,  0.25   ],
        [ 0.2495 , -0.5    ],
        [-0.5    ,  0.2505 ],
        [ 0.25   , -0.499  ],
        [-0.5    ,  0.25   ],
        [ 0.24925, -0.50075],
        [ 0.25   , -0.5    ]],

       [[ 0.25125, -0.49875],
        [-0.5    ,  0.25   ],
        [ 0.2495 , -0.5    ],
        [-0.5    ,  0.2505 ],
        [ 0.25   , -0.499  ],
        [-0.5    ,  0.25   ],
        [ 0.24925, -0.50075],
        [ 0.25   , -0.5    ]]], dtype=float32)>

In [240]:
MPOs=mpg.mean_payoffs(G,S0,S1)
A=np.zeros(n)
B=np.zeros(n)
for u,T in MPOs:
    if T:
        A[u]=MPOs[(u,T)]
    else:
        B[u]=MPOs[(u,T)]
A,B

(array([ 0.25, -0.5 ,  0.25, -0.5 ,  0.25, -0.5 ,  0.25,  0.25]),
 array([-0.5 ,  0.25, -0.5 ,  0.25, -0.5 ,  0.25, -0.5 , -0.5 ]))

In [28]:
import mpg.visualisation.game as v_game
VG=v_game.MPGVisualisation(G)
VG

MPGVisualisation(layout=Layout(height='500px', width='100%'))

In [17]:
from numba import jit,njit,cuda

@njit
def rand_choice_nb(arr, prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    if isinstance(arr,int):
        arr=np.arange(arr)
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

@cuda.jit(["i4(i4,f4[:])"])
def rand_choice_nb_cuda(arr, cum_prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    if isinstance(arr,int):
        arr=np.arange(arr)
    a=0
    b=len(cum_prob)
    c=0
    z=np.random.random_sample()
    while a<b:
        c=(a+b)//2
        if cum_prob[c] > z:
            b=c-1
        else:
            a=c+1
    return arr[c]

@njit(["f8[:](f8[:,:],f8[:,:],f8[:,:],i8,i8,i8)","f4[:](f4[:,:],f4[:,:],f4[:,:],i8,i8,i8)"])
def simulation_mean_payoffs_one(w,s0,s1,iters,depth,n):
    Z=np.zeros(n,dtype=w.dtype)
    for u in range(n):
        s=u
        for k in range(iters):
            for d in range(depth):
                #print(s)
                r=rand_choice_nb(n,s0[s,:] if d%2==0 else s1[s,:])
                Z[u]+=w[s,r]
                s=int(r)
    Z/=iters*depth
    return Z

@cuda.jit
def simulation_mean_payoffs_one_cuda(w,s0,s1,iters,depth,Z,s0_cum,s1_cum):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    n=ty
    m=tx
    s=u
    for k in range(iters):
        for d in range(depth):
            #print(s)
            r=rand_choice_nb_cuda(n,s0_cum[s,:] if d%2==0 else s1_cum[s,:])
            Z[tx,ty]+=w[s,r]
            s=int(r)

from typing import List
import numpy.typing as npt
            
#@njit("float32[:](float32[:,:,:],float32[:,:,:],float32[:,:,:],int32,int32)")
@njit
def simulation_mean_payoffs_impl(W,S0,S1,iters=1000,depth=1000):
    Z=[]
    m=W.shape[0]
    for w,s0,s1 in zip(W,S0,S1):
        Z.append(simulation_mean_payoffs_one(w,s0,s1,iters,depth,n))
        #Z.append(w[1,3])
    return Z
            
def simulation_mean_payoffs(W,S0,S1,iters=1000,depth=1000):
    return tf.stack(simulation_mean_payoffs_impl(W,S0,S1,iters,depth))
    
def simulation_mean_payoffs_cuda(W,S0,S1,iters=1000,depth=1000):
    m=W.shape[0]
    n=W.shape[-1]
    Z=np.zeros((m,n))
    S0_cum=np.cumsum(S0,axis=-1)
    S1_cum=np.cumsum(S1,axis=-1)
    simulation_mean_payoffs_one_cuda[W.shape[0],W.shape[-1]](W,S0,S1,iters,depth,Z,S0_cum,S1_cum)
    Z/=(iters*depth)
    return Z

TypeError: ['i4(i4,f4[:])'] is not a callable object

In [12]:
simulation_mean_payoffs_one(W.astype(np.float64),Z0,Z1,100,100,W.shape[-1])

array([-0.4995,  0.25  , -0.5   ,  0.2502, -0.4996,  0.25  , -0.5003,
       -0.5   ])

In [13]:
import tensorflow as tf
simulation_mean_payoffs_cuda(tf.cast(tf.stack([W,W]),dtype=tf.float64).numpy(),tf.stack([Z0,Z0]).numpy(),tf.stack([Z1,Z1]).numpy(),10,10)

/home/ramizouari/anaconda3/envs/stochastic_games/lib/python3.10/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


TypingError: Failed in cuda mode pipeline (step: nopython frontend)
[1m[1mFailed in cuda mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in method random_sample of numpy.random.mtrand.RandomState object at 0x7f4a200fb740>) found for signature:
 
 >>> random_sample(size=Literal[int](1))
 
There are 4 candidate implementations:
[1m      - Of which 4 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 129.
        With argument(s): '(size=int64)':[0m
[1m       Rejected as the implementation raised a specific error:
         NumbaTypeError: Failed in nopython mode pipeline (step: nopython frontend)
       [1mOnly accept returning of array passed into the function as argument[0m[0m
  raised from /home/ramizouari/anaconda3/envs/stochastic_games/lib/python3.10/site-packages/numba/core/typed_passes.py:150
[0m
[0m[1mDuring: resolving callee type: Fun

In [213]:
W.dtype

dtype('int64')

In [359]:
S0

{1: 4, 2: 3, 4: 5, 3: 6, 7: 1, 6: 1, 5: 7, 0: 1}

In [284]:
@cuda.jit
def rand_choice_nb_cuda(arr, cum_prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    if isinstance(arr,int):
        arr=np.arange(arr)
    a=0
    b=len(cum_prob)
    c=0
    z=np.random.random()
    while a<b:
        c=(a+b)//2
        if cum_prob[c] > z:
            b=c-1
        else:
            a=c+1
    return arr[c]

In [287]:
from numba import cuda
A=np.array([1,2,3])
P=np.array([0.2,0.2,0.6])
A_gpu=cuda.device_array(A)
P_gpu=cuda.device_array(P)
rand_choice_nb_cuda[4,4](A_gpu,P_gpu)

ERROR:numba.cuda.cudadrv.driver:Call to cuInit results in CUDA_ERROR_UNKNOWN


CudaSupportError: Error at driver init: Call to cuInit results in CUDA_ERROR_UNKNOWN (999)

In [162]:
import tensorflow as tf



def expected_mean_payoffs_tensor(W,S0,S1,iters=1000,tol=1e-6):
    R=[]
    for d0,d1 in ([S1,S0],[S0,S1]):
        d=d0@d1
        a=tf.reduce_sum(W*d0,axis=-1,keepdims=True)
        b=tf.reduce_sum(W*d1,axis=-1,keepdims=True)
        q=d0@b+a
        D=tf.eye(W.shape[-1],batch_shape=W.shape[:-2])
        S=tf.zeros(W.shape)
        if iters is not None:
            for i in range(iters):
                S+=D
                D=D@d
            R.append((S@q)/(2*iters))
        else:
            # Perform the eigen decomposition of the one turn transition matrix
            D,V=tf.linalg.eig(d)
            # Compute the mean payoff projection matrix
            # It is a projection matrix into the 1-eigenspace of the one turn transition matrix parallel to the complement eigenspaces
            D_mean=tf.where(tf.abs(D-1)<=tol,tf.constant([1.+0.j],dtype=tf.complex64),tf.constant([0.+0.j],dtype=tf.complex64))
            # Compute the mean payoffs
            Q=tf.math.real(V@tf.linalg.diag(D_mean)@tf.linalg.inv(V))
            R.append(Q@q/2)
    return tf.concat(R,axis=-1)

def mat_power(A,n):
    """
    Compute the matrix power A^n
    :param A: The matrix
    :param n: The power
    """
    if n==0:
        return tf.eye(A.shape[-1],batch_shape=A.shape[:-2])
    elif n==1:
        return A
    elif n%2==0:
        return mat_power(A@A,n//2)
    else:
        return A@mat_power(A@A,n//2)

@tf.function(jit_compile=True)
def expected_discounted_payoffs_tensor(W,S0,S1,gamma,iters=1000,normalize=False):
    R=[]
    with tf.control_dependencies([ tf.Assert(tf.less_equal(tf.abs(gamma),1), [gamma]) ] ):
    for d0,d1 in ([S1,S0],[S0,S1]):
        d=d0@d1
        a=tf.reduce_sum(W*d0,axis=-1,keepdims=True)
        b=tf.reduce_sum(W*d1,axis=-1,keepdims=True)
        q=d0@b+a
        Id=tf.eye(W.shape[-1],batch_shape=W.shape[:-2])
        A=tf.linalg.inv(Id-gamma*d)
        if iters is not None:
            B=Id-mat_power(gamma*d,iters)
            z=A@(B@q)/2
            if normalize:
                z*=(1-gamma)
            R.append(z)
        else:
            z=A@q/2
            if normalize:
                z*=(1-gamma)
            R.append(z)
    return tf.concat(R,axis=-1)


In [191]:
W=tf.cast(W,dtype=tf.float32)
Z0=tf.cast(Z0,dtype=tf.float32)
Z1=tf.cast(Z1,dtype=tf.float32)

expected_discounted_payoffs_tensor(W,Z0,Z1,gamma=3,iters=None,normalize=True)

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[ 4.4868650e+23,  1.4357960e+24],
       [ 1.7947450e+23, -8.9737240e+22],
       [-8.0763535e+23, -7.1789799e+23],
       [ 5.3842349e+23, -2.6921173e+23],
       [-8.0763520e+23, -7.1789814e+23],
       [ 5.3842349e+23, -2.6921175e+23],
       [ 4.4868650e+23,  1.4357960e+24],
       [ 4.4868617e+23,  1.4357960e+24]], dtype=float32)>

In [92]:
expected_mean_payoffs_tensor(W,Z0,T,iters=None)

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[ 2.4999976e-01, -5.9604645e-08],
       [ 1.9032420e-08,  2.5000003e-01],
       [ 2.4999976e-01, -1.1920929e-07],
       [ 4.1974239e-08,  2.5000000e-01],
       [ 2.4999976e-01, -5.9604645e-08],
       [-8.0806544e-09,  2.5000003e-01],
       [ 2.4999976e-01, -1.1920929e-07],
       [ 2.4999976e-01, -1.1920929e-07]], dtype=float32)>

In [91]:
T=A/tf.reduce_sum(A,axis=-1,keepdims=True)
T=tf.cast(T,tf.float32)
T

<tf.Tensor: shape=(8, 8), dtype=float32, numpy=
array([[0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ]], dtype=float32)>

In [114]:
expected_discounted_payoffs_tensor(tf.stack([W,W]),tf.stack([T,Z0]),tf.stack([T,Z1]),0.99,iters=None,normalize=True)

ValueError: in user code:

    File "/tmp/ipykernel_9339/425849310.py", line 49, in expected_discounted_payoffs_tensor  *
        raise ValueError("The discount factor must be in (-1,1)")

    ValueError: The discount factor must be in (-1,1)


In [107]:
expected_mean_payoffs_tensor(tf.stack([W,W]),tf.stack([T,Z0]),tf.stack([T,Z1]),iters=None)

<tf.Tensor: shape=(2, 8, 2), dtype=float32, numpy=
array([[[-2.3841858e-07, -2.3841858e-07],
        [ 3.1403726e-09,  3.1403726e-09],
        [-2.9802322e-07, -2.9802322e-07],
        [ 3.7222669e-09,  3.7222669e-09],
        [-2.3841858e-07, -2.3841858e-07],
        [ 5.1484914e-09,  5.1484914e-09],
        [-2.9802322e-07, -2.9802322e-07],
        [-2.9802322e-07, -2.9802322e-07]],

       [[ 2.4999991e-01, -5.0000036e-01],
        [-4.9999994e-01,  2.5000000e-01],
        [ 2.4999991e-01, -5.0000036e-01],
        [-4.9999994e-01,  2.4999999e-01],
        [ 2.4999997e-01, -5.0000036e-01],
        [-4.9999997e-01,  2.5000000e-01],
        [ 2.4999997e-01, -5.0000012e-01],
        [ 2.4999997e-01, -5.0000012e-01]]], dtype=float32)>